8.1 Sample

In [1]:
var sample = System.IO.File.ReadAllText(@".\sample.txt").Split(Environment.NewLine);
sample

index,value
0,nop +0
1,acc +1
2,jmp +4
3,acc +3
4,jmp -3
5,acc -99
6,acc +1
7,jmp -4
8,acc +6


In [1]:
List<string> allPossible = new List<string>();
var parentNames = new[]{"shiny gold bag"};
int rounds = 0;
Console.Out.WriteLine(String.Join(", ", parentNames));
while(parentNames.Length > 0 && rounds++ < 100)
{
    parentNames = sample.Select(x => x.Split(new[]{" contain ", ", "}, StringSplitOptions.None))
        .Where(x => x.Skip(1).Any(y => parentNames.Any(z => y.Contains(z))))
        .Select(x => x.First().TrimEnd('s'))
        .ToArray();
    Console.Out.WriteLine(rounds + " " + String.Join(", ", parentNames));
    allPossible.AddRange(parentNames);
}
allPossible

shiny gold bag


1 bright white bag, muted yellow bag


2 light red bag, dark orange bag


3 


index,value
0,bright white bag
1,muted yellow bag
2,light red bag
3,dark orange bag


8.1 Input

In [1]:
var input = System.IO.File.ReadAllText(@".\input.txt").Split(Environment.NewLine);
input

index,value
0,nop +283
1,acc +26
2,acc +37
3,acc +6
4,jmp +109
5,acc +10
6,jmp +18
7,acc +5
8,jmp +327
9,acc -4


In [1]:
List<(int i, string newOp)> attempts = input
    .Select((x, i) => (i, newOp:x.StartsWith("jmp") ? "nop" : x.StartsWith("nop") ? "jmp" : x.Substring(0, 3)))
    .Where(x => x.newOp == "jmp" || x.newOp == "nop")
    .ToList();
attempts

index,Item1,Item2
0,0,jmp
1,4,nop
2,6,nop
3,8,nop
4,10,nop
5,13,jmp
6,15,nop
7,17,jmp
8,19,nop
9,20,jmp


In [1]:
var output = display("Starting");

public record Operation(String op, int Value)
{
    public bool Visited {get; private set;}
    public void Visit()
    {
        Visited = true;
    }
};

var maxRounds = 10000;
var tick = 0;
var acc = 0;
var i = 0;
List<(int i, Operation op)> opLog = new List<(int i, Operation op)>();
Operation prevOp;
int prevI;
var prevAttempt = "";

foreach(var attempt in attempts)
{
    output.Update($"Attempt: {attempt}, Prev: {prevAttempt}");

    try
    {
        maxRounds = 10000;
        tick = 0;
        acc = 0;
        i = 0;
        opLog = new List<(int i, Operation op)>();
        prevOp = null;
        prevI = -1;

        var ops = input.Select(x => {
            var parts = x.Split(' ');
            return new Operation(parts[0], Convert.ToInt32(parts[1]));
        }).ToArray();

        ops[attempt.i] = ops[attempt.i] with { op = attempt.newOp };
        while(true && tick++ < maxRounds)
        {
            Func<Operation, int> doAcc = (op) => {
                acc+=op.Value;
                return i + 1;
            };

            var op = ops[i];

            prevOp = op;
            opLog.Add((i, op));
            prevI = i;

            if (op.Visited) {
                // foreach(var log in opLog.Where(x => x.op.op == "jmp" || x.op.op == "nop")) {
                //     Console.WriteLine($"({log.i}, \"{log.op.op}\"),");
                // }
                // Console.WriteLine($"{tick} {i} {op}");
                throw new Exception("Looped");
            }

            i = op switch {
                // {Visited: true} => -1,
                {op: "nop"} => i + 1,
                {op: "acc"} => doAcc(op),
                {op: "jmp"} => i + op.Value,
                _ => throw new Exception("Unknown op")
            };
            // output.Update($"{tick}: {op.op} {op.Value} {op.Visited} > {i} {acc}");

            op.Visit();

            if (i >= ops.Length - 1)
            {
                Console.WriteLine(attempt.i);
                Console.WriteLine("Terminated");
                break;
            }

            if (tick % 100 == 0)
            {
                System.Threading.Thread.Sleep(100);
            }

            if (i < 0 || i >= ops.Length) break;
        }
    }
    catch(Exception ex)
    {
        prevAttempt = ex.Message;
        // Console.WriteLine("Next attempt");
        System.Threading.Thread.Sleep(100);
        continue;
    }
    break;
}

acc

Attempt: (304, nop), Prev: Looped

304


Terminated


1089